In [1]:

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from datetime import datetime, timedelta
from geopy.distance import geodesic as GD 
from math import radians, cos, sin, asin, sqrt

In [2]:
light_on_Merged_FullDataSet = pd.read_csv('../data/MergedData.csv')
light_on = light_on_Merged_FullDataSet.head(9000)

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_50458/117179744.py:1: DtypeWarning: Columns (16,22,23,26,27,29,30,31,32,33,34,35,36,38,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  light_on_Merged_FullDataSet = pd.read_csv('../data/MergedData.csv')


convert columns to appropriate format

In [3]:
cols = ['spn', 'fmi', 'AcceleratorPedal', 'BarometricPressure', 'CruiseControlSetSpeed', 'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature', 'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IntakeManifoldTemperature', 'LampStatus', 'Speed', 'Throttle', 'TurboBoostPressure']
light_on[cols] = light_on[cols].apply(pd.to_numeric, errors='coerce', axis=1)
light_on

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_50458/3486558198.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_on[cols] = light_on[cols].apply(pd.to_numeric, errors='coerce', axis=1)


,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,NaN,False,78.8,1023.0,True,NaN,0.000000,3276.75,NaN,0.00
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,9829,9830,1431047,2015-03-19 04:01:17.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN
8996,9830,9831,1431227,2015-03-19 04:31:28.000,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN
8997,9831,9832,1431235,2015-03-19 04:32:07.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,NaN,True,120.2,1023.0,NaN,NaN,6.524397,3276.75,NaN,3.48
8998,9832,9833,1431260,2015-03-19 04:27:17.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,...,32.0,True,129.2,1279.0,False,NaN,6.534106,3276.75,0.0,2.03


In [4]:
light_on['spn_fmi'] = light_on['spn'].astype(str) + '_' + light_on['fmi'].astype(str)
light_on

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_50458/2423738569.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  light_on['spn_fmi'] = light_on['spn'].astype(str) + '_' + light_on['fmi'].astype(str)


,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,ecuMake,...,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi
0,0,1,990349,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,False,78.8,1023.0,True,NaN,0.000000,3276.75,NaN,0.00,111.0_17.0
1,1,2,990360,2015-02-21 11:34:34.000,NaN,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,629.0_12.0
2,2,3,990364,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,1807.0_2.0
3,3,4,990370,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,NaN,unknown,unknown,unknown,unknown,...,True,NaN,1279.0,NaN,NaN,NaN,NaN,NaN,NaN,1807.0_2.0
4,4,5,990416,2015-02-21 11:39:41.000,NaN,NaN,22281684P01*22357957P01*22362082P01*,13063430,0USA13_13_0415_2238A,VOLVO,...,NaN,NaN,16639.0,NaN,NaN,NaN,NaN,NaN,NaN,4364.0_17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,9829,9830,1431047,2015-03-19 04:01:17.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN,929.0_9.0
8996,9830,9831,1431227,2015-03-19 04:31:28.000,Condition Exists Cruise Control Enable Switch,NaN,unknown,unknown,unknown,unknown,...,NaN,NaN,255.0,NaN,NaN,NaN,NaN,NaN,NaN,596.0_31.0
8997,9831,9832,1431235,2015-03-19 04:32:07.000,Low (Severity Low) Engine Coolant Level,NaN,unknown,unknown,unknown,unknown,...,True,120.2,1023.0,NaN,NaN,6.524397,3276.75,NaN,3.48,111.0_17.0
8998,9832,9833,1431260,2015-03-19 04:27:17.000,Abnormal Update Rate Tire Location,NaN,unknown,unknown,unknown,unknown,...,True,129.2,1279.0,False,NaN,6.534106,3276.75,0.0,2.03,929.0_9.0


In [5]:
# dummy_columns = pd.get_dummies(light_on, columns = ['spn_fmi']) # temporarily commented out in order to reduce file size by removing the dummy columns 
dummy_columns = light_on

In [6]:
dummy_columns['is_fullderate'] = (dummy_columns['spn'] == 5246).astype(int)

/var/folders/nl/hqcw9pqd0ns4q0b61syv0q2m0000gn/T/ipykernel_50458/2470401655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_columns['is_fullderate'] = (dummy_columns['spn'] == 5246).astype(int)


In [7]:
dummy_columns = dummy_columns.sort_values(['EquipmentID', 'EventTimeStamp'])
dummy_columns['is_fullderate_group'] = dummy_columns.groupby('EquipmentID')['is_fullderate'].cumsum()

In [8]:
Num_of_FullDerates_inDataSet = dummy_columns['is_fullderate_group'].sum()
Num_of_FullDerates_inDataSet # 1

np.int64(580)

In [9]:
dummy_columns['EquipID_Index'] = dummy_columns['EquipmentID'].astype(str) + '_' + dummy_columns['is_fullderate_group'].astype(str)

In [10]:
dummy_columns['spn_fmi'] = dummy_columns['spn_fmi'].str.replace('.0', '')

In [11]:
dummy_columns.replace(to_replace='3276,75', value='3278.75', inplace=True)

In [12]:
dummy_columns[['ecuSource', 'LampStatus', 'ecuSerialNumber', 'ecuModel', 'ecuMake']] = dummy_columns[['ecuSource', 'LampStatus', 'ecuSerialNumber', 'ecuModel', 'ecuMake']].astype('category')

In [13]:
dummy_columns[['activeTransitionCount']] = dummy_columns[['activeTransitionCount']].astype('category')

In [16]:
dummy_columns['Latitude'] = dummy_columns['Latitude'].astype(float)

In [17]:
dummy_columns['Longitude'] = dummy_columns['Longitude'].astype(float)

In [18]:
dummy_columns = dummy_columns.sort_values(by=['spn_fmi','EventTimeStamp'], ascending=[True, True]).reset_index()

dummy_columns.to_csv('../data/TwentyRows_Preceding_SPN5246_LightOn_Truck_Identifier_SubsequentTrialRun.csv')

In [19]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees).
    Returns distance in kilometers.
    """
    # Convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of earth in kilometers
    
    return c * r

def find_distance_to_next_condition(df, condition_func, id_column='EquipID_Index', lat_column='Latitude', lon_column='Longitude'):
    """
    For each row in the dataframe, find the next row with the same ID that meets
    a specific condition, and calculate the geographic distance between them.
    
    Parameters:
    - df: pandas DataFrame containing the data
    - condition_func: Function that takes a row and returns True if condition is met
    - id_column: Column name containing equipment IDs
    - lat_column: Column name containing latitude values
    - lon_column: Column name containing longitude values
    
    Returns:
    - DataFrame with original data plus a new column 'Distance_To_Next_Condition_km'
    """
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Add a column for the calculated distances, initialize with NaN
    result_df['Distance_To_Next_Condition_km'] = 9999999999.9
    
    # Group the dataframe by the ID column
    grouped = result_df.groupby(id_column)
    
    # Process each group (each unique equipment ID)
    for equipment_id, group in grouped:
        # Get indices for this group
        indices = group.index.tolist()
        
        # For each row in the group
        for i, idx in enumerate(indices[:-1]):  # Skip the last row as it can't have a "next" row
            current_row = group.loc[idx]
            
            # Find the next row with the same ID that meets the condition
            for next_idx in indices[i+1:]:
                next_row = group.loc[next_idx]
                
                if condition_func(next_row):
                    # Calculate distance between current and matching row
                  #  distance = GD((current_row[lat_column], current_row[lon_column]), (next_row[lat_column], next_row[lon_column])).km
                    
                    haversine_distance(
                        current_row[lat_column], current_row[lon_column],
                        next_row[lat_column], next_row[lon_column]
                    )
                    
                    # Store the calculated distance
                    result_df.at[idx, 'Distance_To_Next_SPN5246_km'] = distance
                    break  # Stop after finding the first matching row
    
    return result_df


In [17]:
def calculate_distance_to_next_spn_5246(df, id_column='EquipID_Index', 
                                        lat_column='Latitude', lon_column='Longitude',
                                        spn_column='spn', target_spn=5246, 
                                        no_match_value=9999999999.9):
    """
    For each row, calculates the geographic distance to the next row with the same
    EquipID_index where spn equals 5246. If no such row exists, assigns 9999999999.9
    
    Parameters:
    - df: pandas DataFrame containing the data
    - id_column: Column name containing equipment IDs
    - lat_column: Column name containing latitude values
    - lon_column: Column name containing longitude values
    - spn_column: Column name containing SPN values
    - target_spn: The specific SPN value to look for (default is 5246)
    - no_match_value: Value to use when no matching row is found
    
    Returns:
    - DataFrame with original data plus a new column 'Distance_To_Next_SPN_5246_km'
    """
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Add a column for the calculated distances, initialize with the no_match_value
    result_df['Distance_To_Next_SPN_5246_km'] = no_match_value
    
    # Group the dataframe by the ID column
    grouped = result_df.groupby(id_column)
    
    # Process each group (each unique equipment ID)
    for equipment_id, group in grouped:
        # Sort the group by its original index to maintain the order
      #  group = group.sort_index()
        
        # Get indices for this group
        indices = group.index.tolist()
        
        # For each row in the group (except the last one)
        for i, idx in enumerate(indices[:-1]):
            current_row = group.loc[idx]
            
            # Find the next row with the same equipment ID where spn equals target_spn
            next_matching_idx = None
            for next_idx in indices[i+1:]:
                next_row = group.loc[next_idx]
                if next_row[spn_column] == target_spn:
                    next_matching_idx = next_idx
                    break
            
            # If we found a matching row, calculate the distance
            if next_matching_idx is not None:
                next_matching_row = group.loc[next_matching_idx]
                distance = GD((current_row[lat_column], current_row[lon_column]), (next_matching_row[lat_column], next_matching_row[lon_column])).km
                
             #   haversine_distance(
             #       current_row[lat_column], current_row[lon_column],
             #       next_matching_row[lat_column], next_matching_row[lon_column]
             #   )
                result_df.at[idx, 'Distance_To_Next_SPN_5246_km'] = distance
            
            # If not, the default no_match_value remains in place
    
    return result_df

In [21]:
Distance_Col_AddedTo_DummyCols = calculate_distance_to_next_spn_5246(df = dummy_columns)

In [22]:
def check_time_intervals(df, EventTimeStamp='EventTimeStamp', condition_column='spn', condition_value=5246, time_interval_column_name='time_to_next_SPN5246'):
    """
    Calculate the time interval between the current row and the next row(s) that meet a specific condition.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        The dataframe containing the timestamp and condition columns.
    EventTimeStamp : str
        The name of the column containing datetime values.
    condition_column : str, optional
        The name of the column to check for the condition. If None, all rows are considered.
    condition_value : any, optional
        The value to check for in the condition column. If None, all rows are considered.
    interval_column_name : str, optional
        The name for the new column that will store the time interval information.
        Default is 'time_to_next_condition'.
    
    Returns:
    --------
    pandas.DataFrame
        The original dataframe with an additional column showing the time interval
        to the next row meeting the condition.
    """
    # Ensure the timestamp column is in datetime format
    df[EventTimeStamp] = pd.to_datetime(df[EventTimeStamp])
    
    # Create a copy of the dataframe to avoid modifying the original
    result_df = df.copy()
    
    # Add a column to store the time intervals with the specified name
    result_df[time_interval_column_name] = timedelta(days=9999999)  # Initialize with a large timedelta
    
    
    # Filter rows that meet the condition if a condition is specified
    if condition_column is not None and condition_value is not None:
        condition_rows = df[df[condition_column] == condition_value]
    else:
        condition_rows = df
    
    # For each row in the dataframe
    for i in range(len(df) - 1):
        current_time = df.iloc[i][EventTimeStamp]
        current_EquipmentID = df.iloc[i]['EquipmentID']
        
        # Find the next row that meets the condition and is after the current row
        next_condition_rows = condition_rows[(condition_rows[EventTimeStamp] > current_time) & (condition_rows['EquipmentID'] == current_EquipmentID)]
      #  prev_condition_rows = condition_rows[(condition_rows[EventTimeStamp] < current_time) & (condition_rows['EquipmentID'] == current_EquipmentID)]

        if not next_condition_rows.empty:
            # Get the earliest next condition time
            next_condition_time = next_condition_rows[EventTimeStamp].min()
            
            # Calculate the time difference
            time_diff = next_condition_time - current_time
            
            # Store the result in the specified column
            result_df.iloc[i, result_df.columns.get_loc(time_interval_column_name)] = time_diff
            
     #   elif not next_condition_rows.empty: 
     #       result_df.iloc[i, result_df.columns.get_loc(time_interval_column_name)] = timedelta(days=8888888)
    
    return result_df


In [23]:
TimeIntervalColumn_AddedTo_DummyColumns = check_time_intervals(Distance_Col_AddedTo_DummyCols, EventTimeStamp='EventTimeStamp', condition_column='spn', condition_value=5246, time_interval_column_name='time_to_next_SPN5246') 
TimeIntervalColumn_AddedTo_DummyColumns

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246
0,368,435,436,996177,2015-02-21 18:50:15,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,9999999 days
1,369,436,437,996181,2015-02-21 18:50:17,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,9999999 days
2,560,673,674,999650,2015-02-22 08:12:30,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,9999999 days
3,562,675,676,999663,2015-02-22 08:12:32,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,9999999 days
4,2358,2682,2683,1021425,2015-02-23 12:44:33,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,9999999 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,8280,8996,8997,1351615,2015-03-14 04:00:41,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,0.0000,3276.75,0.0,0.29,97_15,0,0,1627_0,1.000000e+10,9999999 days
8996,8281,8997,8998,1351708,2015-03-14 04:24:17,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,97_15,0,0,1627_0,1.000000e+10,9999999 days
8997,8371,9105,9106,1366494,2015-03-15 19:19:43,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00033706*082113134117*07700053*I0*BBZ*,79722862,6X1u10D1500000000,...,NaN,NaN,NaN,NaN,97_15,0,0,1437_0,1.000000e+10,9999999 days
8998,1410,1607,1608,1002325,2015-02-22 11:44:29,High (Severity Medium) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,...,65.8168,3276.75,NaN,29.00,97_16,0,0,1429_0,1.000000e+10,9999999 days


In [24]:
def convert_to_hours(delta):
    total_seconds = delta.dt.total_seconds()
    hours = (total_seconds / 3600).astype(float)
    return hours


In [25]:
TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] = convert_to_hours(TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246']) 
TimeIntervalColumn_AddedTo_DummyColumns

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246
0,368,435,436,996177,2015-02-21 18:50:15,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0
1,369,436,437,996181,2015-02-21 18:50:17,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0
2,560,673,674,999650,2015-02-22 08:12:30,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0
3,562,675,676,999663,2015-02-22 08:12:32,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0
4,2358,2682,2683,1021425,2015-02-23 12:44:33,NaN,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,8280,8996,8997,1351615,2015-03-14 04:00:41,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,0.0000,3276.75,0.0,0.29,97_15,0,0,1627_0,1.000000e+10,239999976.0
8996,8281,8997,8998,1351708,2015-03-14 04:24:17,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,NaN,NaN,NaN,NaN,97_15,0,0,1627_0,1.000000e+10,239999976.0
8997,8371,9105,9106,1366494,2015-03-15 19:19:43,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00033706*082113134117*07700053*I0*BBZ*,79722862,6X1u10D1500000000,...,NaN,NaN,NaN,NaN,97_15,0,0,1437_0,1.000000e+10,239999976.0
8998,1410,1607,1608,1002325,2015-02-22 11:44:29,High (Severity Medium) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,...,65.8168,3276.75,NaN,29.00,97_16,0,0,1429_0,1.000000e+10,239999976.0


In [26]:
TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'].min()

np.float64(0.008333333333333333)

In [27]:
TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['spn'] == 5246].index.tolist()


[5019,
 5020,
 5021,
 5022,
 5023,
 5024,
 5025,
 5026,
 5027,
 5028,
 5029,
 5030,
 5031,
 5032,
 5033,
 5034]

In [28]:
target_index = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['spn'] == 5246].index.tolist()

TimeIntervalColumn_AddedTo_DummyColumns.loc[target_index, 'time_to_next_SPN5246'] = 0

In [29]:
time_conditions = [
    TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] == 0,
    (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] <= 2) & (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 0),
    (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 2) & (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] <= 24),
    (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] > 24) & (TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] < 9999999),
    TimeIntervalColumn_AddedTo_DummyColumns['time_to_next_SPN5246'] == 9999999,
]

choices = [3, 2, 1, 0, 9999999]
TimeIntervalColumn_AddedTo_DummyColumns['time_interval_to_SPN5246_class'] = np.select(time_conditions, choices)

In [30]:
target_index_Dist = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['spn'] == 5246].index.tolist()

TimeIntervalColumn_AddedTo_DummyColumns.loc[target_index_Dist, 'Distance_To_Next_SPN5246_km'] = 0

In [31]:
distance_conditions = [
    TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] == 0,
    (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] <= 273.58) & (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] > 0),
    (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] > 273.58) & (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] <= 3282.96),
    (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] > 3282.96) & (TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] < 9999999999.9),
    TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'] == 9999999999.9
]

distance_choices = [3, 2, 1, 0, 9999999999]
TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_class'] = np.select(distance_conditions, distance_choices)

In [32]:
TimeIntervalColumn_AddedTo_DummyColumns['time_interval_to_SPN5246_class'].value_counts()

time_interval_to_SPN5246_class
0    8930
1      43
3      16
2      11
Name: count, dtype: int64

In [33]:
TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_class'].value_counts() 

Distance_To_Next_SPN5246_class
0    8984
3      16
Name: count, dtype: int64

In [43]:
TimeIntervalColumn_AddedTo_DummyColumns['Distance_To_Next_SPN5246_km'].value_counts() 

Distance_To_Next_SPN5246_km
0.0    16
Name: count, dtype: int64

In [34]:
TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['spn'] == 5246]['spn_fmi'].value_counts()

spn_fmi
5246_0    16
Name: count, dtype: int64

In [35]:
TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['time_interval_to_SPN5246_class'] == 3]['spn_fmi'].value_counts()

spn_fmi
5246_0    16
Name: count, dtype: int64

In [36]:
Pre2019_df = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] < datetime(2019, 1, 1)]
Pre2019_df

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246,time_interval_to_SPN5246_class,Distance_To_Next_SPN5246_km,Distance_To_Next_SPN5246_class
0,368,435,436,996177,2015-02-21 18:50:15,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
1,369,436,437,996181,2015-02-21 18:50:17,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
2,560,673,674,999650,2015-02-22 08:12:30,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
3,562,675,676,999663,2015-02-22 08:12:32,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
4,2358,2682,2683,1021425,2015-02-23 12:44:33,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,8280,8996,8997,1351615,2015-03-14 04:00:41,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,0.29,97_15,0,0,1627_0,1.000000e+10,239999976.0,0,NaN,0
8996,8281,8997,8998,1351708,2015-03-14 04:24:17,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,NaN,97_15,0,0,1627_0,1.000000e+10,239999976.0,0,NaN,0
8997,8371,9105,9106,1366494,2015-03-15 19:19:43,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00033706*082113134117*07700053*I0*BBZ*,79722862,6X1u10D1500000000,...,NaN,97_15,0,0,1437_0,1.000000e+10,239999976.0,0,NaN,0
8998,1410,1607,1608,1002325,2015-02-22 11:44:29,High (Severity Medium) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,...,29.00,97_16,0,0,1429_0,1.000000e+10,239999976.0,0,NaN,0


In [37]:
Pre2019_df_AltApproach = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] < pd.Timestamp('2019-01-01')]
Pre2019_df_AltApproach

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246,time_interval_to_SPN5246_class,Distance_To_Next_SPN5246_km,Distance_To_Next_SPN5246_class
0,368,435,436,996177,2015-02-21 18:50:15,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
1,369,436,437,996181,2015-02-21 18:50:17,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
2,560,673,674,999650,2015-02-22 08:12:30,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
3,562,675,676,999663,2015-02-22 08:12:32,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
4,2358,2682,2683,1021425,2015-02-23 12:44:33,NaN,NaN,unknown,unknown,unknown,...,NaN,0_0,0,0,1620_0,1.000000e+10,239999976.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,8280,8996,8997,1351615,2015-03-14 04:00:41,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,0.29,97_15,0,0,1627_0,1.000000e+10,239999976.0,0,NaN,0
8996,8281,8997,8998,1351708,2015-03-14 04:24:17,High (Severity Low) Water In Fuel Indicator,NaN,unknown,unknown,unknown,...,NaN,97_15,0,0,1627_0,1.000000e+10,239999976.0,0,NaN,0
8997,8371,9105,9106,1366494,2015-03-15 19:19:43,High (Severity Low) Water In Fuel Indicator,NaN,04993120*00033706*082113134117*07700053*I0*BBZ*,79722862,6X1u10D1500000000,...,NaN,97_15,0,0,1437_0,1.000000e+10,239999976.0,0,NaN,0
8998,1410,1607,1608,1002325,2015-02-22 11:44:29,High (Severity Medium) Water In Fuel Indicator,NaN,04993120*00022630*082113134117*07700053*I0*BBZ*,79463845,6X1u10D1500000000,...,29.00,97_16,0,0,1429_0,1.000000e+10,239999976.0,0,NaN,0


In [38]:
Pre2019_df.to_csv('../data/Pre2019_df_Distance.csv')

In [39]:
In2019_df = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] > datetime(2018, 12, 31)]
In2019_df

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246,time_interval_to_SPN5246_class,Distance_To_Next_SPN5246_km,Distance_To_Next_SPN5246_class


In [40]:
In2019_df_AltApproach = TimeIntervalColumn_AddedTo_DummyColumns[TimeIntervalColumn_AddedTo_DummyColumns['EventTimeStamp'] > pd.Timestamp('2018-12-31')]
In2019_df_AltApproach

,index,Unnamed: 0,RecordID,ESS_Id,EventTimeStamp,eventDescription,actionDescription,ecuSoftwareVersion,ecuSerialNumber,ecuModel,...,TurboBoostPressure,spn_fmi,is_fullderate,is_fullderate_group,EquipID_Index,Distance_To_Next_SPN_5246_km,time_to_next_SPN5246,time_interval_to_SPN5246_class,Distance_To_Next_SPN5246_km,Distance_To_Next_SPN5246_class


In [41]:
In2019_df.to_csv('../data/In2019_df_Distance.csv')

In [42]:
TimeIntervalColumn_AddedTo_DummyColumns.to_csv('../data/MergedFullDataSet_TimeIntervalLabeled_PerTruck_Distance.csv')